In [1]:
import requests#Connect to endpoints
import time #time script
import sys
import csv
import pandas as pd
import spotipy#authentication
import spotipy.util as util#authentication
from spotipy.oauth2 import SpotifyClientCredentials#authentication

In [2]:
#Credentials
cid = '049ade7215e54c63a2b628f3784dc407'
secret = 'b86bedbebf53486b970e92b2cb7b612c'
redirect_uri = 'http://google.com/'
username = 'xxx'

In [3]:
#Authentication
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
def get_album_track(url,artist_uri):
    all_albums = []
    try:
        resp = requests.get(url=url,
                            headers={'Authorization': 'Bearer ' + token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
        
    response = resp.json()
    
    album_limit = (response['limit'])-1
    
    for x in range(album_limit):
        try:
            track_uri = response['items'][x]['uri'].split(':')
            track_name = response['items'][x]['name'].replace('[','').replace(']','')
            print(track_uri,track_name)
            all_albums.append({'track_uri':track_uri[2],'track_name':track_name})
        except IndexError as error:
            continue #Natural part of the process. The length of response is sometimes less than limit
    try:
        if (url is not None):
            get_albums(response['next'])
        else:
            return
    except:
        return all_albums

In [15]:
i = 0
track_list=[]

start = time.time()
with open('Data/artist_album_uri.csv') as csvfile:
    file = csv.reader(csvfile, delimiter='|')
    next(file)
    head = [next(file) for x in range(50)] #for limiting amount of rows read
    
    for row in head:
        artist_uri = row[0]
        artist_name = row[1]
        album_uri = row[2]
        print(i,artist_uri,artist_name,album_uri)
        albumtracks = get_album_track('https://api.spotify.com/v1/albums/{id}/tracks?limit=14'.format(id=album_uri),artist_uri)#list of album uris
        for track in albumtracks:
            track_list.append({'Artist URI':artist_uri,'Artist Name':artist_name,'Album URI':album_uri
                               ,'track_uri':track['track_uri'],'track_name':track['track_name']})#output album_uri
        
        i+=1
        
stop = time.time()
duration = (stop - start)/60

0 3bBWKHfpepPOychRNFzg4q Al Di Meola 7xbyqbmIudUFYHsgHEvcwp
['spotify', 'track', '7cVA2h2HRlE2mOxhXEmtro'] Here Comes the Sun
['spotify', 'track', '5N6IdYnIYaATgQKHg9CZJP'] Golden Slumbers
['spotify', 'track', '2Fz3MpPkiUPEH6d4Heuipp'] Dear Prudence
['spotify', 'track', '4yDo6bylEHQMbAaXLKSvwl'] Norwegian Wood
['spotify', 'track', '1kgKtxtHLvRPrmvX8ns79t'] Mother Nature's Son
['spotify', 'track', '2f9m1gEeEveK3E4nkXOvD3'] Strawberry Fields Forever
['spotify', 'track', '1yaj9IUWz7QaBpuwYxRsp1'] Yesterday
['spotify', 'track', '1M9aXWCWRtmiuUJuDNMUyg'] Your Mother Should Know
['spotify', 'track', '2V3KpCCtGjjCJ1MUnKid89'] Hey Jude
['spotify', 'track', '3tYOJ81C6qHIIkW8FAU69e'] I'll Follow the Sun
['spotify', 'track', '2wBMA1jDkLAwcotlwgRATP'] Julia
['spotify', 'track', '3hmkXA8pT2eT08wVM8md28'] Till There Was You
['spotify', 'track', '2WWXgrorazoGDKb4NJ0fMJ'] Here, There and Everywhere
1 3bBWKHfpepPOychRNFzg4q Al Di Meola 0HSSxy1xsVQNMobVl0Cr37
['spotify', 'track', '6Ow6K4WkOQBdXo1KAicr4Q

['spotify', 'track', '32sPaXoEqIHoE7WjgvFHWd'] Milonga Noctiva
['spotify', 'track', '4OvVBDWEEJMaCduTvQ6Kts'] Broken Heart
['spotify', 'track', '2MEIBDTGeHr6jkyDge5zjH'] Ava's Dream Sequence Lullaby
['spotify', 'track', '2ybpUrPzIKWxC16gzWJTzX'] Cerreto Sannita
['spotify', 'track', '2woP1kEMiFqKl3JCONl1UQ'] Notorious
['spotify', 'track', '70siAkCdGf9KfZjUYpuCkA'] Frozen in Time
['spotify', 'track', '7tdk45H4H6JdGo1nFud7z7'] Escapado
['spotify', 'track', '38pfrZwnJA4C4tb0ziZIib'] Pomp
['spotify', 'track', '2eThHoUoS7bjr2RzqeU3nJ'] Left unsaid
['spotify', 'track', '4ed1eMN8e8rihgbZDKvy5w'] Insieme
['spotify', 'track', '2Xiwy6E4TvwfaP6rZX8K9z'] Rebels
12 3bBWKHfpepPOychRNFzg4q Al Di Meola 1p6nqxRgbOvQWCR3lfRota
['spotify', 'track', '3brfF2lcznoUfbKxg2gVvS'] Milonga Noctiva
['spotify', 'track', '230rjMpjwZLmBvxZ06BbJa'] Broken Heart
['spotify', 'track', '5xLM8NkX4df94ORh03Mj5l'] Ava's Dream Sequence Lullaby
['spotify', 'track', '4cUHG9Q6gwloePG7icuDOv'] Cerreto Sannita
['spotify', 'track',

['spotify', 'track', '0vnqfSi4iiytfkA1of6dcV'] Guajira Flamenca - Remastered 2014
['spotify', 'track', '2M4vO68puGdgTFmsSA7nc9'] Anda Jaleo - Remastered 2014
['spotify', 'track', '3x2SS3Iqa5YUJr0K6UeVsj'] Impetu - Remastered 2014
['spotify', 'track', '2ZHZeZ4HiL2u7SerNIDAEL'] Panaderos Flamencos - Remastered 2014
['spotify', 'track', '67CNow4oC2gIkfVX7fYh8I'] Aires De Linares - Remastered 2014
['spotify', 'track', '2lgNY4RrsE9yhckVQ6xe52'] Y Tú No Me Respondías - Remastered 2014
['spotify', 'track', '6FFhysI9AYVp5a1HWbGKqD'] Malaguena De Lecuona - Remastered 2014
['spotify', 'track', '6H0lHmco4FkdMvcsgLmNzJ'] Rumba Improvisada - Remastered 2014
['spotify', 'track', '4WO5q4gmBiPJbv2xqVDADE'] Entre Dos Aguas - Remastered 2014
['spotify', 'track', '7uuoCYATtH8LQ349jxa8oy'] Fuente Y Caudal - Remastered 2014
['spotify', 'track', '6HSTbVShLcPjOqXQOBfDK6'] Plaza Alta - Remastered 2014
['spotify', 'track', '3aODu7Id15q8e88XIbRDsR'] Monasterio De Sal - Remastered 2014
['spotify', 'track', '78BM

['spotify', 'track', '1MstnOkmxEbeQ0FKs4hZms'] Variaciones De Minera (Minera) - Live In Spain/2010
['spotify', 'track', '3Iek4iBrRnffvk1ifeppMz'] Mi Antonia (Bulería Por Solea) - Live In Spain/2010
['spotify', 'track', '3NoyMloctv4lGT49cdx2jb'] Tangos Con Cositas Buenas (Tangos) - Live In Spain/2010
['spotify', 'track', '0LhepUVALaSjDy9ukUHYHh'] Moraito Siempre (Bulería) - Live In Spain/2010
['spotify', 'track', '0gKv5HHcJihCCinAEXB3rn'] El Cafetal (Rumba) - Live In Spain/2010
['spotify', 'track', '6UFzHPKcRyEUSLhXUrZEqj'] Lagartijo (Siguirilla) - Live In Spain/2010
['spotify', 'track', '4A9ZQM74E3SojHvmnBGE5M'] Zyryab En Vivo - Live In Spain/2010
['spotify', 'track', '39QWVIMGulnNwd7KbvGktF'] Vámonos - Live In Spain / 2010
34 3h8OjAdgYXVRoMZ8jFd6Uw Paco de Lucía 0UbsOp3YYszvXnR1EQ8pBG
['spotify', 'track', '7CuB8jrvnjMqUXjdPHiuDY'] Variaciones De Minera (Minera) - Live In Spain/2010
['spotify', 'track', '5CMwAHkqHPVmde93K3PLHi'] Mi Antonia (Bulería Por Solea) - Live In Spain/2010
['spo

['spotify', 'track', '7FqapucqvR0NKYbbJkYqQw'] Dancing in the Clouds
['spotify', 'track', '4VnKLwZWcKq1AhnXMjL6FU'] Guitarra Poeta
['spotify', 'track', '4Q36KopyeaTwjlpJIuc4eP'] Romance De La Luna
['spotify', 'track', '714NLLiNnAK5Cb4po0KrlB'] Lagrimas
['spotify', 'track', '2FKULTeHW1ypAkYtjdO1fM'] Hearts Desire
['spotify', 'track', '5KLYeZGXkQKUzOTGCTIeju'] Roses in the Wind
['spotify', 'track', '5DiH6NYPp2E9fEsyHAeHNC'] Gypsy Waltz
['spotify', 'track', '1W97DUwHLv827ZKf9zyElW'] Sentimientos De Amor
['spotify', 'track', '2H3Hf0adoo999RU8MgFDt1'] When I First Saw You
['spotify', 'track', '3E9DQ8qnsUERzstnaaIMLH'] Moonlight Stroll
['spotify', 'track', '1uktY7MhgNKrPGYl6k8XJi'] Poeta
46 2Z8XcWdgy5dSaiQR3d7tHK Armik 3RcYE9xDqPYK0hHQD32mXy
['spotify', 'track', '7cZKtSBE1jU2FEo617TeBr'] Alegra
['spotify', 'track', '5WOsIvsp684MXy4Ll3naAM'] Summer Night's Dance
['spotify', 'track', '2KNSRgUmK9vdFLYgB76ttW'] Luz del Sol
['spotify', 'track', '5rs88tL8HK51mmEYKK3ba6'] Dream Catcher
['spotify', 

In [16]:
#Duration of script
str(duration)+' minutes'

'0.0955893596013387 minutes'

In [40]:
df = pd.DataFrame(track_list)
df
#Test whether there are duplicates for Album URI
df_dropduplicates = df.drop_duplicates(['Artist URI', 'track_name'])
df_dropduplicates
df_dropduplicates.to_csv('Data/artist_album_track_uri.csv',sep='|',index=False)

In [18]:
track_list

[{'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'Al Di Meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '7cVA2h2HRlE2mOxhXEmtro',
  'track_name': 'Here Comes the Sun'},
 {'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'Al Di Meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '5N6IdYnIYaATgQKHg9CZJP',
  'track_name': 'Golden Slumbers'},
 {'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'Al Di Meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '2Fz3MpPkiUPEH6d4Heuipp',
  'track_name': 'Dear Prudence'},
 {'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'Al Di Meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '4yDo6bylEHQMbAaXLKSvwl',
  'track_name': 'Norwegian Wood'},
 {'Artist URI': '3bBWKHfpepPOychRNFzg4q',
  'Artist Name': 'Al Di Meola',
  'Album URI': '7xbyqbmIudUFYHsgHEvcwp',
  'track_uri': '1kgKtxtHLvRPrmvX8ns79t',
  'track_name': "Mother Nature's Son"},
 {'Artist URI': '3bBWKHfpepPOychRNFz